In [2]:
import numpy as np
import pandas as pd
import statistics 

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding, Bidirectional, GRU, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
with open('links.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
links = texts.split('\n')
    

In [4]:
len_links = pd.Series([len(i.split('/')) - 1 for i in links])
"среднее значение длины {0}, мода {1}, максимальное значение {2}, медиана {3}".format(statistics.mean(len_links), 
                                                                                     statistics.mode(len_links), 
                                                                                     len_links.max(),
                                                                                     statistics.median(len_links))

'среднее значение длины 3.8947035239502004, мода 3, максимальное значение 10, медиана 4'

In [19]:
links[0].split('/')

['https:', '', 'www.youtube.com', 'channel', 'UCzKpUpv903t7aT4_TN0rfPg']

In [5]:
data = tokenizer.texts_to_sequences(links)
#res = np.array( data[0])
x = pad_sequences(data, maxlen=6, padding='post')
x[:5]

NameError: name 'tokenizer' is not defined

In [5]:
df = pd.DataFrame(columns=['start_domain', 'domain', 'end_domain', 'dictionary'])
df

,start_domain,domain,end_domain,dictionary


In [6]:
for i in links:
    link = i.split('/')
    entire_domain = link[2].split('.')
    if 'com' in entire_domain:
        index_domain = entire_domain.index('com') - 1
        if entire_domain[-1] == 'com':
            end_domain = entire_domain[-1]
        else:
            end_domain = ".".join(entire_domain[index:])
    else:
        index_domain = len(entire_domain) - 2
        end_domain = entire_domain[-1]
    if len(df.loc[df['domain'] == entire_domain[index_domain]]) != 0:
        index = df.loc[df['domain']==entire_domain[index_domain]].index[0]
        df.loc[index,'dictionary'].extend(link[3:])
        if index_domain != 0:
            for sd in entire_domain[:index_domain]:
                if sd not in df.loc[index,'start_domain']:
                    df.loc[index,'start_domain'].append(sd)
        else: 
            if '' not in df.loc[index,'start_domain']:
                    df.loc[index,'start_domain'].append(sd)
        if end_domain not in df.loc[index,'end_domain']:
            df.loc[index,'start_domain'].append(end_domain)
    else:
        new_row = {'start_domain':entire_domain[:index_domain], 'domain':entire_domain[index_domain], 
                   'end_domain': end_domain, 'dictionary':link[3:]}
        df = df.append(new_row, ignore_index=True)


new_row = {'initial_part_domain':['f'], 'domain':'a', 'dictionary':['ff']}
df = df.append(new_row, ignore_index=True)
len(df[df['domain'].str.contains('a', case=False)])

In [7]:
pd.set_option('display.max_rows', None)
df

,start_domain,domain,end_domain,dictionary
0,"[www, m, artists, socialimpact, tv]",youtube,com,"[channel, UCzKpUpv903t7aT4_TN0rfPg, user, bart..."
1,"[workspace, fonts, firebase, www, admob, cloud...",google,com,"[, , , , home, , opencue, create, photobooks, ..."
2,"[fanyi, mtc, openauth, shouji, online, carlife...",baidu,com,"[, , , software, , , , , view, f83f578383d0496..."
3,"[ac, weread, sports, docs, gameinstitute, gouh...",qq,com,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
4,"[developers, socialimpact, www]",facebook,com,"[, , ubcessa, , LifeLineEkur, , playcoralsprin..."
5,"[izhongchou, world, daxue, helpcenter, www, zc...",taobao,com,"[, , , , m, , , m, maijia, qingdan, 2005359829..."
6,"[logistics, music, advertising, aws, www, sell...",amazon,com,"[, , , , luna, events, , primeday, amazonprime..."
7,"[finance, tw, stock, sports, search, news, www...",yahoo,com,"[quote, MO, , markets, currency-converter, , q..."
8,"[daily, www, zhuanlan]",zhihu,com,"[story, 9745703, question, 405440459, p, 47170..."
9,"[sw, en, id, pl, ru, ja, sv, tr, es, zh, ca, a...",wikipedia,org,"[wiki, Baraza_la_mawaziri_Tanzania, wiki, Gun_..."


In [8]:
#убрать пустые строки
#index = df.loc[df['domain']=='a'].index[0]
words = []
for index in range(df.shape[0]):
    df.loc[index,'dictionary'] = list(filter(lambda x: x != "", df.loc[index,'dictionary']) )
    words.extend(df.loc[index,'dictionary'])
df

,start_domain,domain,end_domain,dictionary
0,"[www, m, artists, socialimpact, tv]",youtube,com,"[channel, UCzKpUpv903t7aT4_TN0rfPg, user, bart..."
1,"[workspace, fonts, firebase, www, admob, cloud...",google,com,"[home, opencue, create, photobooks, about, new..."
2,"[fanyi, mtc, openauth, shouji, online, carlife...",baidu,com,"[software, view, f83f578383d049649b665831, ite..."
3,"[ac, weread, sports, docs, gameinstitute, gouh...",qq,com,"[367958257, ent, 10002730, tft, nba, tv, accel..."
4,"[developers, socialimpact, www]",facebook,com,"[ubcessa, LifeLineEkur, playcoralsprings, bran..."
5,"[izhongchou, world, daxue, helpcenter, www, zc...",taobao,com,"[m, m, maijia, qingdan, 200535982937.htm, qing..."
6,"[logistics, music, advertising, aws, www, sell...",amazon,com,"[luna, events, primeday, amazonprime, healthla..."
7,"[finance, tw, stock, sports, search, news, www...",yahoo,com,"[quote, MO, markets, currency-converter, quote..."
8,"[daily, www, zhuanlan]",zhihu,com,"[story, 9745703, question, 405440459, p, 47170..."
9,"[sw, en, id, pl, ru, ja, sv, tr, es, zh, ca, a...",wikipedia,org,"[wiki, Baraza_la_mawaziri_Tanzania, wiki, Gun_..."


In [9]:
words

['channel',
 'UCzKpUpv903t7aT4_TN0rfPg',
 'user',
 'bartletttv',
 'channel',
 'UCsB7IPUQGVpAmn8C9ZXWT7A',
 'channel',
 'UCS9Krr7eb0pEa1wS8EVIFwA',
 'user',
 'mozilla',
 'channel',
 'UCG26bSkEjJc7SqGsxoHNnbA',
 'user',
 'SingingWalrusMusic',
 'user',
 'Betaradesgrstoixima',
 'user',
 'MuseumJewishHeritage',
 'channel',
 'UCw4X4WsJHT7UA_XhPONv5VQ',
 'channel',
 'UCYp3rk70ACGXQ4gFAiMr1SQ',
 'user',
 'algundiaalgunaparte',
 'user',
 'TheVoiceKidsAU',
 'user',
 'TravellerYT',
 'c',
 'MrMindBlow',
 'user',
 'usponline',
 'channel',
 'UCxJmOWf__YcLgB19gy5dGQQ',
 'user',
 'LegalTalkNetwork',
 'user',
 'thecreativepenn',
 'c',
 'StudentsUnionUCL',
 'c',
 'ResearchShorts',
 'c',
 'Eduty',
 'user',
 'AnAmericanHomestead',
 'c',
 'JackMasseyWelsh',
 'user',
 'vanentrepreneurgroup',
 'c',
 'RumboM%25C3%25A9dico',
 'intl',
 'ro',
 'user',
 'CornellLawSchool',
 'channel',
 'UCDnwlb3IQDPJtFysPUJbDFQ',
 'user',
 'EdisonBOE',
 'user',
 'HeinzCollege',
 'user',
 'transportgovuk',
 'c',
 'my9nj',
 'user',

In [10]:
maxWordsCount = 2000
tokenizer = Tokenizer(num_words=maxWordsCount, 
                      lower=True, char_level=False)
tokenizer.fit_on_texts(words)
tokenizer

In [11]:
dist = list(tokenizer.word_counts.items())
dist.sort(key = lambda x: (x[1], x[0]), reverse=True)
print(dist[:10])
#tokenizer.word_index

[('25d8', 6107), ('25d9', 3775), ('html', 2811), ('htm', 2120), ('25a7', 1580), ('citations', 1424), ('3fuser', 1412), ('26hl', 1388), ('3den', 1365), ('25db', 1302)]


In [12]:
x = []
for i in links:    
    link = i.split('/')
    entire_domain = link[2].split('.')
    if 'com' in entire_domain:
        index_domain = entire_domain.index('com') - 1
    else:
        index_domain = len(entire_domain) - 2
    domain = entire_domain[index_domain]
    x.append([df.loc[df['domain']==entire_domain[index_domain]].index[0]])
x = np.array(x)
x

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [45],
       [45],
       [45]], dtype=int64)

In [14]:
inp_words = 4
y = []
for i in links:
    link = i.split('/')
    parts = link[3:]
   
    y.append(parts)
    
y = tokenizer.texts_to_sequences(y)
for i in range(len(y)):
    if len(y[i]) < inp_words:
        if len(y[i]) == 0:
            y[i] = [1] * inp_words
        else: 
            while len(y[i]) < inp_words: 
                y[i].extend(y[i])
            y[i] = y[i][:inp_words]
    elif len(y[i]) > inp_words:
        y[i] = y[i][:inp_words]
y

[[269, 269, 269, 269],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [91, 366, 91, 366],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [158, 158, 158, 158],
 [158, 158, 158, 158],
 [269, 269, 269, 269],
 [158, 158, 158, 158],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [198, 198, 198, 198],
 [269, 269, 269, 269],
 [158, 158, 1

In [15]:
#y = [to_categorical(i, num_classes=maxWordsCount) for i in y]
y = to_categorical(y, num_classes=maxWordsCount)
y

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]], dtype=float32)

In [16]:
#train, test = train_test_split(x, test_size = 0.2, random_state = 17)
n = len(x) // 5 * 4
train_x, train_y = x[:n], y[:n]
test_x, test_y= x[n:], y[n:]
train_x

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [39],
       [39],
       [39]], dtype=int64)

In [44]:
train_y.shape

(26536, 4, 2000)

In [45]:
train_x.shape

(26536, 1)

In [21]:
len(train_y)

26536

In [ ]:
# x(количество доменов, максиму словаря, 1)
# y(домены, словарь?, 2000)

In [17]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = 2))
#model.add(GRU(128, activation='relu'))
model.add(Bidirectional(GRU(100, activation='relu'
                            , return_sequences=True,
                           input_shape=(x.shape[0], x.shape[1]))))
#model.add(TimeDistributed(Dense(4)))
#model.add(SimpleRNN(128, activation='tanh'))
model.add(TimeDistributed(Dense(maxWordsCount, activation='softmax')))
model.summary()

#loss='categorical_crossentropy'
model.compile(loss='mse', metrics=['accuracy'], optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 256)            512000    
                                                                 
 bidirectional (Bidirectiona  (None, 2, 200)           214800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 2, 2000)          402000    
 ibuted)                                                         
                                                                 
Total params: 1,128,800
Trainable params: 1,128,800
Non-trainable params: 0
_________________________________________________________________


In [18]:
history = model.fit(train_x, train_y, batch_size=32, epochs=10)


Epoch 1/10
830/830 [==============================] - 10s 9ms/step - loss: 4.9969e-04 - accuracy: 0.3887
Epoch 2/10
830/830 [==============================] - 8s 9ms/step - loss: 4.7526e-04 - accuracy: 0.4259
Epoch 3/10
830/830 [==============================] - 8s 9ms/step - loss: 3.4550e-04 - accuracy: 0.4259
Epoch 4/10
830/830 [==============================] - 8s 9ms/step - loss: 3.4507e-04 - accuracy: 0.4259
Epoch 5/10
830/830 [==============================] - 8s 9ms/step - loss: 3.4474e-04 - accuracy: 0.4259
Epoch 6/10
830/830 [==============================] - 8s 9ms/step - loss: 3.3974e-04 - accuracy: 0.4361
Epoch 7/10
830/830 [==============================] - 8s 9ms/step - loss: 3.1281e-04 - accuracy: 0.4991
Epoch 8/10
830/830 [==============================] - 8s 9ms/step - loss: 2.9758e-04 - accuracy: 0.5297
Epoch 9/10
830/830 [==============================] - 8s 9ms/step - loss: 2.9065e-04 - accuracy: 0.5444
Epoch 10/10
830/830 [==============================] - 8s 9ms/s

In [47]:
def buildPhrase(texts, str_len=6):
    res = '/'.join(texts)
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)
        print(indx)
        print(tokenizer.index_word[indx])
        res += "/" + tokenizer.index_word[indx]  # дописываем строку
    l_array = res.split('/')
    index_end = len(l_array) + 1 if l_array.count('https') == 1 else l_array.index('https', 1)
    index_com = l_array.index('com') + 1
    link = 'https://' + '.'.join(l_array[1:index_com]) + '/' + '/'.join(l_array[index_com:index_end])
    return link



In [48]:
res = buildPhrase(['youtube'])
#res = buildPhrase("https/www/youtube")
print(res)

[0 0 0 ... 0 0 0]


TypeError: unhashable type: 'numpy.ndarray'

In [19]:
y_pred = model.predict(test_x)
y_pred.shape

#accuracy_score(test_y, y_pred)

(6637, 1, 2000)

In [20]:
y_pred

array([[[1.4135117e-06, 9.8152405e-01, 1.1894700e-06, ...,
         1.4797464e-06, 1.1053465e-06, 1.3212560e-06]],

       [[1.4135117e-06, 9.8152405e-01, 1.1894700e-06, ...,
         1.4797464e-06, 1.1053465e-06, 1.3212560e-06]],

       [[1.4135117e-06, 9.8152405e-01, 1.1894700e-06, ...,
         1.4797464e-06, 1.1053465e-06, 1.3212560e-06]],

       ...,

       [[2.5828378e-04, 2.8980201e-01, 2.5732099e-04, ...,
         2.7407956e-04, 2.3794443e-04, 2.6887644e-04]],

       [[2.5828378e-04, 2.8980201e-01, 2.5732099e-04, ...,
         2.7407956e-04, 2.3794443e-04, 2.6887644e-04]],

       [[2.5828384e-04, 2.8980181e-01, 2.5732105e-04, ...,
         2.7407962e-04, 2.3794449e-04, 2.6887652e-04]]], dtype=float32)